# Create A Genesis Branch And Submit

First change to root directory

In [1]:
import os 
os.chdir('..')

Load some rpc gadgets to send calls

In [2]:
from utils.rpc import lib as rpc_lib
from utils.encode.bytes import encode_bytes_to_base64_str
from config.branch_cfg import TWIG_BRANCH_TYPE_ID
from config.bucket_cfg import BUCKET_ID_TYPE_NO_REF, BUCKET_ID_TYPE_WITH_ID_REF, DEFAULT_MOLECULAR_BUCKET_SCHEMA, DEFAULT_ATOMIC_BUCKET_SCHEMA

In [3]:
# Some useful constants
BYTES0 = encode_bytes_to_base64_str(bytes(0))

In [4]:
# Some useful print functions
def print_response_in_human_readable_format(response):
    print("The branch id is {branch_id}.\nThere were {nr_buckets} buckets submitted (same order as in the contents argument):{refs}.\nThe following Article Names have been registered:{names}.\nThe submit id (hash) is {submit_id}.\nThe submit trace id is {submit_trace_id}.".format(
        **response, 
        nr_buckets=len(response["bucket_refs"]),
        refs=("\n\t" if response["bucket_refs"] else "") + ",\n\t".join(response["bucket_refs"]),
        names=("\n\t" if response["registered_names"] else "") + ",\n\t".join([f"{n['name']} (at {n['id']})" for n in response['registered_names']])))

### Create Genesis Branch
Make the parameters for a genesis branch creation

In [5]:
# some fake signature encoded in bytes64 
signature = BYTES0
accept_conflicts = False
msg = 'Genesis Submit'
name = 'Genesis Branch'
kwargs = dict(
    branch_type=TWIG_BRANCH_TYPE_ID, 
    name='Genesis Branch', 
    signature=signature, 
    accept_conflicts=True, 
    msg='Genesis Submit')

In [6]:
create_genesis_response = rpc_lib.json_rpc_call(method='create_genesis_branch', params=kwargs)
print(f"The response is the branch_id: {create_genesis_response['result']}")

The response is the branch_id: AVESCMio/7qCyofj


In [7]:
branch_id = create_genesis_response['result']

### Send a first submit
Make the parameters for the first submit

In [8]:
contents = [
        {   
            ## A CONTENT BUCKET FOR AN ARTICLE
            "data": "=Hello=",  
            "schema": DEFAULT_ATOMIC_BUCKET_SCHEMA, 
            "parent_id": BYTES0, 
            "signature": BYTES0, 
            "refs": []
        },
        {
            ## ANOTHER CONTENT BUCKET FOR AN ARTICLE
            "data": "=World=",  
            "schema": DEFAULT_ATOMIC_BUCKET_SCHEMA, 
            "parent_id": BYTES0, 
            "signature": BYTES0, 
            "refs": []
        },
        {
            ## A CONTEXT BUCKET (MOLECULAR) FOR THAT ARTICLE
            "data": {
                "order": [
                    {"id": 0, "type": BUCKET_ID_TYPE_NO_REF},
                    {"id": 1, "type": BUCKET_ID_TYPE_NO_REF}], 
                "name": "Dummy Article Name"},
            "schema": DEFAULT_MOLECULAR_BUCKET_SCHEMA,
            "parent_id": BYTES0, 
            "signature": BYTES0, 
            "refs": []
        },
        {
            ## A CONTENT BUCKET FOR ANOTHER ARTICLE
            "data": "=About Lakat=\nContent of 'About Lakat'",  
            "schema": DEFAULT_ATOMIC_BUCKET_SCHEMA, 
            "parent_id": BYTES0, 
            "signature": BYTES0, 
            "refs": []
        },
        {
            ## A CONTEXT BUCKET (MOLECULAR) FOR THAT OTHER ARTICLE
            "data": {
                "order": [{"id": 3, "type": BUCKET_ID_TYPE_NO_REF}], 
                "name": "Lakat"},
            "schema": DEFAULT_MOLECULAR_BUCKET_SCHEMA,
            "parent_id": BYTES0, 
            "signature": BYTES0, 
            "refs": []
        }
    ]
## NOTE: The order of the contents is important. 

In [9]:
# the submit keyword arguments
submit_kwargs = dict(
    branch_id=branch_id, 
    contents=contents, 
    public_key=BYTES0, 
    proof=BYTES0, 
    msg="First Submit")

# make the call
first_submit_response = rpc_lib.json_rpc_call(method="submit_content_to_twig", params=submit_kwargs)

# branch_response
first_submit_response_data = first_submit_response['result']

In [10]:
print_response_in_human_readable_format(first_submit_response_data)

The branch id is AVESCNNQaRo+cgJy.
There were 5 buckets submitted (same order as in the contents argument):
	AVESCGp7iZczl9/AAQ==,
	AVESCODub3fbyGe3AQ==,
	AVESCH5cjfo3Iat4Ag==,
	AVESCCu3ilHmiDblAQ==,
	AVESCEuQDYZqyJs6Ag==.
The following Article Names have been registered:
	Dummy Article Name (at AVESCH5cjfo3Iat4Ag==),
	Lakat (at AVESCEuQDYZqyJs6Ag==).
The submit id (hash) is AVESCOI4G4JHcfX+DQae5XUAAAA=.
The submit trace id is AVESCGwfahQlxfg/Dgae5XUAAAA=.


In [11]:
bucket_ids = first_submit_response_data["bucket_refs"]

### Make another submission

In [12]:
second_contents = [
        {
            ## CHANGE =World= TO =Universe=
            "data": "=Universe=",  
            "schema": DEFAULT_ATOMIC_BUCKET_SCHEMA, 
            "parent_id": bucket_ids[1], # the id of the second bucket, the one we want to change
            "signature": BYTES0, 
            "refs": []
        },
        {
            ## ALSO UPDATE THE CONTEXT BUCKET (MOLECULAR) FOR THAT ARTICLE
            ## NOTE: The second entry of the "order" has a reference type
            "data": {
                "order": [
                    {"id": 0, "type": BUCKET_ID_TYPE_NO_REF},
                    {"id": bucket_ids[1], "type": BUCKET_ID_TYPE_WITH_ID_REF},
                    {"id": 2, "type": BUCKET_ID_TYPE_NO_REF}], # NOTE: id can also be after this id.
                "name": ""}, # NOTE: no need to supply a name anymore (without effect)
            "schema": DEFAULT_MOLECULAR_BUCKET_SCHEMA,
            "parent_id": bucket_ids[2], ## Enter here the parent id of the article 
            "signature": BYTES0, 
            "refs": []
        },
        {
            ## Add another section to the previous article.
            "data": "=Authors=\nThe creator",  
            "schema": DEFAULT_ATOMIC_BUCKET_SCHEMA, 
            "parent_id": BYTES0, 
            "signature": BYTES0, 
            "refs": []
        }
    ]
## NOTE: The order of the contents is important. 

In [13]:
# the submit keyword arguments
second_submit_kwargs = dict(
    branch_id=branch_id, 
    contents=second_contents, 
    public_key=BYTES0, 
    proof=BYTES0, 
    msg="Second Submit")

# make the call
second_submit_response = rpc_lib.json_rpc_call(method="submit_content_to_twig", params=second_submit_kwargs)

# branch_response
second_submit_response_data = second_submit_response['result']

In [14]:
print_response_in_human_readable_format(second_submit_response_data)

The branch id is AVESCNNQaRo+cgJy.
There were 3 buckets submitted (same order as in the contents argument):
	AVESCFMUFpQjAsRAAQ==,
	AVESCI0zGlhdWFHVAg==,
	AVESCMjLJ6YAcmDJAQ==.
The following Article Names have been registered:.
The submit id (hash) is AVESCGzYdrzqMebRDQae5XUAAAA=.
The submit trace id is AVESCNSoNHl5WtMYDgae5XUAAAA=.


### Article Getters

Get Article From Article Name

In [15]:
# get_article_from_article_name returns the latest version of the article
rpc_kwargs = dict(branch_id=branch_id, name='Dummy Article Name') 
get_article_response = rpc_lib.json_rpc_call(method="get_article_from_article_name", params=rpc_kwargs)
get_article_response_data = get_article_response["result"]
print(get_article_response_data)


=Universe=
=World=
=Authors=
The creator


In [16]:
# get_article_from_id returns the article at the given id
get_article_from_id_kwargs = dict(bucket_id=first_submit_response_data["bucket_refs"][2]) 
get_article_from_article_id_response = rpc_lib.json_rpc_call(
    method="get_article_from_article_id", params=get_article_from_id_kwargs)
get_article_from_article_id_response_data = get_article_from_article_id_response["result"]
print(get_article_from_article_id_response_data)


=Hello=
=World=


In [17]:
get_branch_data_kwargs = dict(
    branch_id=branch_id, 
    deserialize_buckets=True) 
branch_data_response = rpc_lib.json_rpc_call(
    method="get_branch_data_from_branch_id", params=get_branch_data_kwargs)
branch_data_response_data = branch_data_response["result"]

In [18]:
branch_data_response_data["submit_trace"]["new_buckets"]

[{'schema_id': 1,
  'signature': '',
  'public_key': '',
  'parent_bucket': 'AVESCODub3fbyGe3AQ==',
  'root_bucket': 'AVESCODub3fbyGe3AQ==',
  'data': '=Universe=',
  'refs': [],
  'timestamp': 1705751583},
 {'schema_id': 2,
  'signature': '',
  'public_key': '',
  'parent_bucket': 'AVESCH5cjfo3Iat4Ag==',
  'root_bucket': 'AVESCH5cjfo3Iat4Ag==',
  'data': ['AVESCFMUFpQjAsRAAQ==',
   'AVESCODub3fbyGe3AQ==',
   'AVESCMjLJ6YAcmDJAQ=='],
  'refs': [],
  'timestamp': 1705751583},
 {'schema_id': 1,
  'signature': '',
  'public_key': '',
  'parent_bucket': '',
  'root_bucket': '',
  'data': '=Authors=\nThe creator',
  'refs': [],
  'timestamp': 1705751583}]

### Inspect Buckets

In [19]:
original_article_id = first_submit_response_data["bucket_refs"][2]
updated_article_id = second_submit_response_data["bucket_refs"][1]

In [20]:
updated_article_id = second_submit_response_data["bucket_refs"][1]
get_bucket_kwargs = dict(bucket_id=updated_article_id) 
bucket_data_response = rpc_lib.json_rpc_call(
    method="get_bucket_from_bucket_id", params=get_bucket_kwargs)
bucket_data_response_data = bucket_data_response["result"]

In [21]:
print('original_article_id:\n', original_article_id) 
print('\nupdated_article_id:\n', updated_article_id)
print('\nget_bucket_kwargs:\n', get_bucket_kwargs)
print('\nbucket_data_response_data:\n', bucket_data_response_data)

original_article_id:
 AVESCH5cjfo3Iat4Ag==

updated_article_id:
 AVESCI0zGlhdWFHVAg==

get_bucket_kwargs:
 {'bucket_id': 'AVESCI0zGlhdWFHVAg=='}

bucket_data_response_data:
 {'schema_id': 2, 'signature': '', 'public_key': '', 'parent_bucket': 'AVESCH5cjfo3Iat4Ag==', 'root_bucket': 'AVESCH5cjfo3Iat4Ag==', 'data': ['AVESCFMUFpQjAsRAAQ==', 'AVESCODub3fbyGe3AQ==', 'AVESCMjLJ6YAcmDJAQ=='], 'refs': [], 'timestamp': 1705751583}


In [22]:
get_bucket_head_kwargs = dict(
    branch_id=branch_id,
    bucket_id=updated_article_id, ## NOTE: this is the id of the updated article
    deserialize_bucket=True) 
bucket_head_data_response = rpc_lib.json_rpc_call(
    method="get_bucket_head_from_bucket_id", params=get_bucket_head_kwargs)
bucket_head_data_response_data = bucket_head_data_response["result"]
print(bucket_head_data_response_data)

{'id': 'AVESCI0zGlhdWFHVAg==', 'data': {'schema_id': 2, 'signature': '', 'public_key': '', 'parent_bucket': 'AVESCH5cjfo3Iat4Ag==', 'root_bucket': 'AVESCH5cjfo3Iat4Ag==', 'data': ['AVESCFMUFpQjAsRAAQ==', 'AVESCODub3fbyGe3AQ==', 'AVESCMjLJ6YAcmDJAQ=='], 'refs': [], 'timestamp': 1705751583}}


In [23]:
get_bucket_head_kwargs = dict(
    branch_id=branch_id,
    bucket_id=original_article_id,  ## NOTE: We use the original article id
    deserialize_bucket=True) 
bucket_head_data_response = rpc_lib.json_rpc_call(
    method="get_bucket_head_from_bucket_id", params=get_bucket_head_kwargs)
bucket_head_data_response_data = bucket_head_data_response["result"]
print(bucket_head_data_response_data)

{'id': 'AVESCI0zGlhdWFHVAg==', 'data': {'schema_id': 2, 'signature': '', 'public_key': '', 'parent_bucket': 'AVESCH5cjfo3Iat4Ag==', 'root_bucket': 'AVESCH5cjfo3Iat4Ag==', 'data': ['AVESCFMUFpQjAsRAAQ==', 'AVESCODub3fbyGe3AQ==', 'AVESCMjLJ6YAcmDJAQ=='], 'refs': [], 'timestamp': 1705751583}}
